In [23]:
from scripts.mapwidget import CustomMap
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
import ipywidgets as ui
import ipyleaflet

map_wid = CustomMap("720px","614px")

map_wid.world_copy_jump = True

mapbox=ui.VBox(children=[map_wid])
#marker=Marker(location=(39.5, -98.35))
#map_wid.add_layer(marker)


#def on_location_changed(event):
#    #map_wid._update_value(marker.location[0],marker.location[1])
#    #print(marker.location)


#marker.observe(on_location_changed,'location')
mapbox




In [25]:
#Shp files
import pandas as pd
import geopandas as gpd
from ipyleaflet import GeoData,Choropleth
import branca.colormap as cm
import fiona 
import numpy as np
import json
import branca.colormap as cm


shp_file = gpd.read_file('/home/mygeohub/sachdevg/SimpleGTool/job/1/outputs/results/Land/Regional/p_QLANDrl/PCT/p_QLANDrl_irrigated.shp')
map_df=shp_file.to_crs(4326)
print(map_df)
min_df = min(map_df['DATA'])
max_df = max(map_df['DATA'])
range_=  max_df - min_df
min_ = min(map_df['DATA']) + (float(50)/ float(100)) * range_
max_ = min(map_df['DATA']) + (float(100)/ float(100)) * range_
map_df = map_df[map_df['DATA'] > min_]
map_df = map_df[map_df['DATA'] < max_]
print(map_df)
temp_file = "p_QLANDrl_irrigated.geojson"
map_df.to_file(temp_file, driver='GeoJSON')
with open(temp_file, 'r') as f:
            geo_json_data = json.load(f)
            #Region based data extraction
            for d in geo_json_data["features"]:
               d["REG"] = d["properties"]["REG"]
print(geo_json_data)
#Taking out Region Vs Value dictionary to map out a color scheme based on number values
mapping  = dict(zip(map_df["REG"].str.strip(), map_df["DATA"]))
print(mapping)
#Creating the color scheme to match the legend
linear = cm.LinearColormap([(255,0,0),(255,51,0),(255,119,0),(255,187,0),(255,255,0),(204,255,0),(153,255,0),(102,255,0),(38,191,0),(0,102,0)],
                 vmin=min(map_df['DATA']), vmax=max(map_df['DATA']))
print(linear)
display(linear)
#display(linear) #Display the color scheme
#The Choropleth widget is used to add color to the regions
if(min(map_df['DATA']) == max(map_df['DATA'])):
             layer = GeoData(geo_dataframe=map_df,style = {'color':'green','fillColor':'rgb(0,102,0)','fillOpacity':0.7})
else:
             layer = Choropleth(
                    geo_data=geo_json_data,
                    choro_data=mapping,
                    colormap=linear,
                    style={'fillOpacity': 0.7, "color":"black"},
                    key_on="REG"
                    )
#print(layer.colormap)
map_wid.add_layer(layer)
map_wid._legend_bar.refresh(min(map_df['DATA']), max(map_df['DATA']))
#print(mapping)

        REG  REGID  GLOBAL       DATA  \
0    AUS_NZ    5.0       3   3.740965   
1    C_Asia   16.0       1   2.162192   
2       CAN   11.0       2  10.127440   
3   CC_Amer    8.0       2   5.369062   
4   CHN_MNG   13.0       1   2.911087   
5    E_Euro    1.0       1   8.060392   
6        EU    6.0       1   8.344039   
7    JPN_KR   15.0       1   3.942632   
8    M_East   14.0       1   5.110504   
9     N_Afr    2.0       1   3.272772   
10    S_Afr    9.0       1   3.504016   
11   S_Amer    4.0       2   4.388526   
12   S_Asia    7.0       1   3.988725   
13  SE_Asia   10.0       1   7.891411   
14      SSA    3.0       1   4.829054   
15       US   12.0       2   2.231482   

                                             geometry  
0   MULTIPOLYGON (((167.50000 -46.50000, 168.00000...  
1   POLYGON ((72.00000 37.00000, 72.00000 36.50000...  
2   MULTIPOLYGON (((-54.00000 48.00000, -53.50000 ...  
3   MULTIPOLYGON (((-67.50000 18.50000, -66.00000 ...  
4   MULTIPOLYGON (((10

In [2]:
map_wid.close()

In [4]:
from scripts.layerservice import RasterLayerUtil
from model.variableutil import VariableModel

id_str = "3"
system_component="Environment"
spatial_resolution = "Geospatial"
type_of_result = "PCT"
result_to_view = "Irrigated"
filter_min = 0
filter_max = 100

variable_model = VariableModel(id_str, system_component, spatial_resolution, type_of_result,result_to_view, filter_min, filter_max,"N-use Grid",1)

if variable_model.is_raster():
                layer_util = RasterLayerUtil(variable_model)
                layer = layer_util.create_layer()
                map_wid.visualize_raster(layer, layer_util.processed_raster_path)
elif variable_model.is_vector():
                layer_util = VectorLayerUtil(variable_model)
                layer = layer_util.create_layer()

/home/mygeohub/sachdevg/SimpleGTool/job/3/outputs/results/Environment/Geospatial/p_QNITROgl/PCT/p_QNITROgl_irrigated.tif
filtering succeeded
min, max (-0.7606434226036072, 47.05888366699219)
tile folder /home/mygeohub/sachdevg/SimpleGTool/job/3/outputs/results/Environment/Geospatial/p_QNITROgl/PCT/p_QNITROgl_irrigated
8
/home/mygeohub/sachdevg/SimpleGTool/job/3/outputs/results/Environment/Geospatial/p_QNITROgl/PCT/p_QNITROgl_irrigated_temp_color.tiff
/home/mygeohub/sachdevg/SimpleGTool/job/3/outputs/results/Environment/Geospatial/p_QNITROgl/PCT/p_QNITROgl_irrigated
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


In [2]:
from osgeo import gdal
# open raster and choose band to find min, max
raster = r'/home/mygeohub/sachdevg/SimpleGTool/job/8/outputs/results/Environment/Geospatial/p_QNLEACHha_gl/PCT/p_QNLEACHha_gl_rainfed.tif'

gtif = gdal.Open(raster)
srcband = gtif.GetRasterBand(1)

# Get raster statistics
stats = srcband.GetStatistics(True, True)

# Print the min, max, mean, stdev based on stats index
print ("[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % (
    stats[0], stats[1], stats[2], stats[3]))

[ STATS ] =  Minimum=-50001.582, Maximum=994.020, Mean=4.572, StdDev=181.984


In [ ]:
from scripts.layerservice import RasterLayerUtil
from model.variableutil import VariableModel

id_str = "8"
system_component="Production"
spatial_resolution = "Geospatial"
type_of_result = "PCT"
result_to_view = "irrigated"
filter_min = 0
filter_max = 100

variable_model = VariableModel(id_str, system_component, spatial_resolution, type_of_result,result_to_view, filter_min, filter_max,"",0)

print(variable_model.file_path().parent)

In [ ]:
import base64
import ipywidgets

def download_button(buffer, filename: str, button_description: str):

    """Loads data from buffer into base64 payload embedded into a HTML button. Recommended for small files only.

    buffer: open file object ready for reading.
        A file like object with a read method.
    filename:    str
        The name when it is downloaded.
    button_description: str
        The text that goes into the button.

    """

    payload = base64.b64encode(buffer).decode()

    html_button = f"""<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" >
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:auto">{button_description}</button>
    </a>
    </body>
    </html>
    """
    return ipywidgets.HTML(html_button)

with open('SIMPLE_G_CornSoy.cmf', 'r') as f:
    file_content = f.read()
    
f = file_content
#print(f)
download_button(bytes(f,"utf-8"), 'SIMPLE_G_CornSoy.cmf','Downloadasdasdasdasdadadsdasda')

In [ ]:
import sqlite3
import os
import subprocess
database = "/data/groups/simpleggroup/job/job.db"
conn = sqlite3.connect(database)
cursor = conn.cursor()
cursor. execute("SELECT * from SIMPLEJobs")
print(cursor. fetchall())
names = [description[0] for description in cursor.description]
print(names)

In [ ]:
import sqlite3
import os
import subprocess
dbfile =os.popen("echo $HOME").read().rstrip('\n') + "/SimpleGTool/DatabaseFile(DONOTDELETE).db"
conn = sqlite3.connect(dbfile)
cursor = conn.cursor()
#Database is always created first so the next statement should not give an error
cursor.execute("SELECT * FROM SIMPLEJobs")
rows = cursor.fetchall()
#Storing the contents of the db in list_of_jobs
list_of_jobs = []
cursor.close()
conn.close()
print(rows)
submit = subprocess.run(["submit", "--status" ,"00119287" ], capture_output=True)
output = submit.stdout.decode("utf-8")
print(submit.stdout)

In [ ]:
import subprocess
#jobcmd = "submit -w 15 -i SIMPLE_G_AllCrops.cmf simpleg_us_all --detach"
submit = subprocess.check_output(["submit", "-w","15","--detach","-i", "SIMPLE_G_AllCrops.cmf","simpleg_us_all"],cwd="/home/mygeohub/sachdevg/SimpleGTool")#, capture_output=True,cwd="/home/mygeohub/sachdevg/SimpleGTool")
print(submit)

In [ ]:
print(submit.stdout.decode("utf-8"))
get_id = submit.stdout.decode("utf-8")
start = get_id.find("Run") + 4
end = get_id.find("registered") -1
print(get_id[start:end])

In [6]:
from scripts.SIMPLEUtil import SIMPLEUtil
import json
import sqlite3
import os, datetime
import sys
from pathlib import Path

dbfile =os.popen("echo $HOME").read().rstrip('\n') + "/SimpleGTool" + '/DatabaseFile(DONOTDELETE).db'

con = sqlite3.connect(dbfile)
conn = con.cursor()
sql = 'delete from SIMPLEJobs where jobid = ?'
cur = conn.execute(sql,("7"))
con.commit()
con.close()

In [ ]:
#Shp files
import pandas as pd
import geopandas as gpd
from ipyleaflet import GeoData,Choropleth
import branca.colormap as cm
import fiona 
import numpy as np
import json
import branca.colormap as cm


shp_file = gpd.read_file('/home/mygeohub/sachdevg/SimpleGTool/job/1/outputs/results/Production/Global/p_PCROP/PCT/p_PCROP.shp')
map_df=shp_file.to_crs(4326)

csv_data = pd.read_csv("/home/mygeohub/sachdevg/SimpleGTool/job/1/outputs/results/Production/Global/p_PCROP/PCT/p_PCROP.csv")
#print(type(map_df))
map_df.to_file("p_QLANDrl_irrigated.geojson", driver='GeoJSON')
with open('p_QLANDrl_irrigated.geojson', 'r') as f:
    geo_json_data = json.load(f)
    for d in geo_json_data["features"]:
       d["REG"] = d["properties"]["REG"]

#print(geo_json_data)
print(csv_data)
mapping  = dict(zip(map_df["REG"].str.strip(), map_df["DATA"]))
linear = cm.LinearColormap([(255,0,0),(255,51,0),(255,119,0),(255,187,0),(255,255,0),(204,255,0),(153,255,0),(102,255,0),(38,191,0),(0,102,0)],
                           vmin=min(map_df['DATA']), vmax=max(map_df['DATA']))
display(linear)
print(map_df)
#print(list(mapping.items())[:10])
#print(geo_json_data['features'][0])
layer = Choropleth(
    geo_data=geo_json_data,
    choro_data=mapping,
    colormap=linear,
    style={'fillOpacity': 1.0, "color":"black"},
    key_on="REG"
)
print(layer.colormap)
map_wid.add_layer(layer)
map_wid._legend_bar.refresh(min(map_df['DATA']), max(map_df['DATA'])/0.9)
print(mapping)

In [ ]:
map_wid.remove_layer(map_wid.layers[-1])

In [ ]:
import branca.colormap as cm
linear = cm.LinearColormap([(255,0,0),(255,51,0),(255,119,0),(255,187,0),(255,255,0),(204,255,0),(153,255,0),(102,255,0),(38,191,0),(0,102,0)],
                           vmin=0, vmax=10)

linear

In [ ]:
import ipywidgets as ui

def _rgb_to_hex( r: int, g: int, b: int):
        assert 0 <= r <= 255
        assert 0 <= g <= 255
        assert 0 <= b <= 255
        return "#{:02x}{:02x}{:02x}".format(r,g,b)

color =  _rgb_to_hex(255, 119, 0)
print(color)
legend_layout = ui.Layout(     
                               width= "fit_content",
                               height= "fit_content",
                               flex_flow= "row",
                               align_items= "center",
                               justify_content= "center",
                               button_color= color,
                               padding= "0px 0px 0px 0px",
                               opacity= "0.75"
                           )
text = ui.Button(description="Test", layout=legend_layout, style = ui.ButtonStyle(button_color=color))

bucket = ui.VBox(children=[text], layout = legend_layout)
bucket

In [ ]:
import os,sys,re

SRC_DIR = os.path.abspath(os.path.join
                            (os.path.dirname(os.path.abspath(__file__)), os.pardir))

In [ ]:
import os

print(os.popen("whoami").read())

In [ ]:
#import subprocess 
#subprocess.Popen(r'c:\mytool\tool.exe', cwd=r'd:\test\local')
import ipywidgets as ui
myupload = ui.FileUpload(
    accept= '.txt', 
    multiple=False  
)
myupload

In [ ]:
list(myupload.value.keys())[0]

In [ ]:
uploaded_filename = next(iter(myupload.value))
content = myupload.value[uploaded_filename]['content']
with open('/home/mygeohub/sachdevg/SimpleGTool/SIMPLE_G_AllCrops.cmf', 'w') as f: f.write(content.decode("utf-8"))